In [36]:
#Importing Necessary Libraries and Creating the Spark Application
from pyspark.sql import SparkSession,functions as func
from pyhive import hive

sparkApp = SparkSession.builder \
    .appName("Spark Proj") \
    .enableHiveSupport() \
    .config("hive.metastore.uris", "thrift://localhost:10000") \
    .getOrCreate()


In [37]:
df = sparkApp.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://localhost:3306/bigdataproj") \
    .option("dbtable", "transaction") \
    .option("user", "root") \
    .option("password", "") \
    .load()

In [38]:
df.show()

+---+-------------+----------+-----------+--------------+--------------+-------------------+-----------------+--------------------+----------+-------------------+------------+----------------+--------------------+-----------+--------------+---------------+-----------------------+-------+----------+---------------+--------------------+----------+------------+-------------+-----------+-------------+----------------+------------------------+-------+
| id|accountNumber|customerId|creditLimit|availableMoney|currentBalance|transactionDateTime|transactionAmount|        merchantName|acqCountry|merchantCountryCode|posEntryMode|posConditionCode|merchantCategoryCode|cardPresent|currentExpDate|accountOpenDate|dateOfLastAddressChange|cardCVV|enteredCVV|cardLast4Digits|     transactionType|echoBuffer|merchantCity|merchantState|merchantZip|posOnPremises|recurringAuthInd|expirationDateKeyInMatch|isFraud|
+---+-------------+----------+-----------+--------------+--------------+-------------------+------

In [39]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- accountNumber: string (nullable = true)
 |-- customerId: string (nullable = true)
 |-- creditLimit: decimal(10,2) (nullable = true)
 |-- availableMoney: decimal(10,2) (nullable = true)
 |-- currentBalance: decimal(10,2) (nullable = true)
 |-- transactionDateTime: timestamp (nullable = true)
 |-- transactionAmount: decimal(10,2) (nullable = true)
 |-- merchantName: string (nullable = true)
 |-- acqCountry: string (nullable = true)
 |-- merchantCountryCode: string (nullable = true)
 |-- posEntryMode: string (nullable = true)
 |-- posConditionCode: string (nullable = true)
 |-- merchantCategoryCode: string (nullable = true)
 |-- cardPresent: boolean (nullable = true)
 |-- currentExpDate: string (nullable = true)
 |-- accountOpenDate: date (nullable = true)
 |-- dateOfLastAddressChange: date (nullable = true)
 |-- cardCVV: string (nullable = true)
 |-- enteredCVV: string (nullable = true)
 |-- cardLast4Digits: string (nullable = true)
 |-- trans

In [40]:
#Columns with only NULL Values, Does not contribute to the EDA process due to Data Loss
COLUMNS_TO_DROP = ['echoBuffer','merchantState','merchantZip','recurringAuthInd','posOnPremises','merchantCity']

df = df.drop(*COLUMNS_TO_DROP)

In [41]:
#Schema of the dataset
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- accountNumber: string (nullable = true)
 |-- customerId: string (nullable = true)
 |-- creditLimit: decimal(10,2) (nullable = true)
 |-- availableMoney: decimal(10,2) (nullable = true)
 |-- currentBalance: decimal(10,2) (nullable = true)
 |-- transactionDateTime: timestamp (nullable = true)
 |-- transactionAmount: decimal(10,2) (nullable = true)
 |-- merchantName: string (nullable = true)
 |-- acqCountry: string (nullable = true)
 |-- merchantCountryCode: string (nullable = true)
 |-- posEntryMode: string (nullable = true)
 |-- posConditionCode: string (nullable = true)
 |-- merchantCategoryCode: string (nullable = true)
 |-- cardPresent: boolean (nullable = true)
 |-- currentExpDate: string (nullable = true)
 |-- accountOpenDate: date (nullable = true)
 |-- dateOfLastAddressChange: date (nullable = true)
 |-- cardCVV: string (nullable = true)
 |-- enteredCVV: string (nullable = true)
 |-- cardLast4Digits: string (nullable = true)
 |-- trans

In [42]:
df.show(5)

+---+-------------+----------+-----------+--------------+--------------+-------------------+-----------------+-------------------+----------+-------------------+------------+----------------+--------------------+-----------+--------------+---------------+-----------------------+-------+----------+---------------+---------------+------------------------+-------+
| id|accountNumber|customerId|creditLimit|availableMoney|currentBalance|transactionDateTime|transactionAmount|       merchantName|acqCountry|merchantCountryCode|posEntryMode|posConditionCode|merchantCategoryCode|cardPresent|currentExpDate|accountOpenDate|dateOfLastAddressChange|cardCVV|enteredCVV|cardLast4Digits|transactionType|expirationDateKeyInMatch|isFraud|
+---+-------------+----------+-----------+--------------+--------------+-------------------+-----------------+-------------------+----------+-------------------+------------+----------------+--------------------+-----------+--------------+---------------+-----------------

In [43]:
#Dropping customerId as it is equivalent to accountNumber
df=df.drop('customerId')

df.show(5)

+---+-------------+-----------+--------------+--------------+-------------------+-----------------+-------------------+----------+-------------------+------------+----------------+--------------------+-----------+--------------+---------------+-----------------------+-------+----------+---------------+---------------+------------------------+-------+
| id|accountNumber|creditLimit|availableMoney|currentBalance|transactionDateTime|transactionAmount|       merchantName|acqCountry|merchantCountryCode|posEntryMode|posConditionCode|merchantCategoryCode|cardPresent|currentExpDate|accountOpenDate|dateOfLastAddressChange|cardCVV|enteredCVV|cardLast4Digits|transactionType|expirationDateKeyInMatch|isFraud|
+---+-------------+-----------+--------------+--------------+-------------------+-----------------+-------------------+----------+-------------------+------------+----------------+--------------------+-----------+--------------+---------------+-----------------------+-------+----------+-------

1) Current Balance is the amount of money spent from the available limit
2) Available Money is the amount of money you're allowed to spend before reaching credit limit

In [44]:
#Handling NULL Values in 'acqCountry' column by filling them using Mode
df.groupBy('acqCountry').count().show()

#Acquiring the mode value and storing it in a variable
acqCountryMode = df.select(func.mode("acqCountry")).first()[0]

#Filling NULL values in the 'acqCountry' using the Mode Value
dfNoNull = df.fillna({'acqCountry':acqCountryMode})

+----------+-----+
|acqCountry|count|
+----------+-----+
|        CA|  156|
|        ME|  208|
|        US|49258|
|          |  264|
|        PR|  113|
+----------+-----+



Too many transactions in a little time is more likely to get the transactions flagged as fraudlent
Spending more money than the amount in 'availableMoney' column results in a fraudlent transaction

In [45]:
#Sample of Fradulent Cases
dfNoNull.filter(func.col('isFraud')=='true').show(10)

+----+-------------+-----------+--------------+--------------+-------------------+-----------------+--------------------+----------+-------------------+------------+----------------+--------------------+-----------+--------------+---------------+-----------------------+-------+----------+---------------+---------------+------------------------+-------+
|  id|accountNumber|creditLimit|availableMoney|currentBalance|transactionDateTime|transactionAmount|        merchantName|acqCountry|merchantCountryCode|posEntryMode|posConditionCode|merchantCategoryCode|cardPresent|currentExpDate|accountOpenDate|dateOfLastAddressChange|cardCVV|enteredCVV|cardLast4Digits|transactionType|expirationDateKeyInMatch|isFraud|
+----+-------------+-----------+--------------+--------------+-------------------+-----------------+--------------------+----------+-------------------+------------+----------------+--------------------+-----------+--------------+---------------+-----------------------+-------+----------+-

Fraud Transactions can be caused as a result of inserting different expiry dates each transaction, none of which match the correct expiration date.
 

In [46]:
incorrectExpiryDateCases = dfNoNull.select('expirationDateKeyInMatch').filter(func.col('isFraud')=='true').groupBy('expirationDateKeyInMatch').count().show()

+------------------------+-----+
|expirationDateKeyInMatch|count|
+------------------------+-----+
|                   false|  683|
+------------------------+-----+



In [47]:
#Exploring Recent Transactions of a Random User
dfNoNull.filter(func.col('accountNumber')=='114896048').show(5)

+---+-------------+-----------+--------------+--------------+-------------------+-----------------+-------------+----------+-------------------+------------+----------------+--------------------+-----------+--------------+---------------+-----------------------+-------+----------+---------------+---------------+------------------------+-------+
| id|accountNumber|creditLimit|availableMoney|currentBalance|transactionDateTime|transactionAmount| merchantName|acqCountry|merchantCountryCode|posEntryMode|posConditionCode|merchantCategoryCode|cardPresent|currentExpDate|accountOpenDate|dateOfLastAddressChange|cardCVV|enteredCVV|cardLast4Digits|transactionType|expirationDateKeyInMatch|isFraud|
+---+-------------+-----------+--------------+--------------+-------------------+-----------------+-------------+----------+-------------------+------------+----------------+--------------------+-----------+--------------+---------------+-----------------------+-------+----------+---------------+---------

Fraudlent Cases are more likely to occur in the absence of the physical card (i.e online payments)

In [48]:
#Correlation Between Presence of Card and Fraudlent Cases
cardPresentCases = dfNoNull.select('cardPresent').filter(func.col('isFraud')=='true').groupBy('cardPresent').count().show()

+-----------+-----+
|cardPresent|count|
+-----------+-----+
|       true|  189|
|      false|  494|
+-----------+-----+



Fraud Activites are most likely to occur in online retail stores

In [49]:
casesPerCategory = dfNoNull.select('merchantCategoryCode').filter(func.col('isFraud')=='true').groupBy('merchantCategoryCode').count().show()

+--------------------+-----+
|merchantCategoryCode|count|
+--------------------+-----+
|              hotels|    5|
|                auto|   11|
|           rideshare|  124|
|                food|   61|
|        online_gifts|   58|
|              health|    3|
|       personal care|   50|
|           furniture|    7|
|       online_retail|  280|
|             airline|   10|
|       entertainment|   17|
|            fastfood|   52|
|       subscriptions|    5|
+--------------------+-----+



USA have witnessed the most amount of fraudlent cases

In [50]:
casesPerAcqCountry = dfNoNull.select('acqCountry').filter(func.col('isFraud')=='true').groupBy('acqCountry').count().show()

+----------+-----+
|acqCountry|count|
+----------+-----+
|        CA|    2|
|        ME|    3|
|        US|  667|
|          |    7|
|        PR|    4|
+----------+-----+



Most Fraudlent Cases when Card Details is being entered manually

In [51]:
casesPerEntryMode = dfNoNull.select('posEntryMode').filter(func.col('isFraud')=='true').groupBy('posEntryMode').count().show()

+------------+-----+
|posEntryMode|count|
+------------+-----+
|          09|  316|
|          05|  128|
|          90|   16|
|          02|  192|
|            |   16|
|          80|   15|
+------------+-----+

